# Study case: renovation project for a building
we apply here the algorithm in 'algorithm.ipynb' to our study case in the picture (see README.md).

#### Package import

In [ ]:
import random
import numpy as np
import pandas as pd

#### Inputs
Here you can change the values for investment costs, number of building elements and the m number, which is defined for the normalized problem as the maximum count of measures over all building elements.

In [ ]:
# The total renovation investment costs
Inv = 5000
# the total number of building elements
n = 4
# the maximum count of measures over all building elements for a normalized problem 
m = 5

#### Parameters of the measures 
Values related to material costs and heat loss costs after 20 years of usage are set randomly for experimental purposes. Their values can also be loaded from other data sources, such as CSV-files.

##### Experimental runs 
As no real values are available, I use hereafter experimental, sampled values:
- Material costs are taken with 1 decimal point and generated using samples from a uniform distribution from low value of 50 to the highest value of 300 price unit
- Heat loss costs after 20 years of usage from 10 to 1000 units

In [ ]:
d = {"e": [], "Kij": [], "Hij": []}
for i in range(n):
    d["e"].append(i)
    # Material costs
    d["Kij"].append(np.round(np.random.uniform(low=50, high=300, size=(m-1,)), decimals=1))
    # Heat loss costs TODO("Rethink low and high values")
    d["Hij"].append(np.round(np.random.uniform(low=10, high=1000, size=(m,)), decimals=1))

#### Step 1:

As mentioned in the Condition 2 in README.md file, the renovation of a building element can be left out, limitation either due to no sufficient investment amount or because the impact of leaving the building element without any renovation is irrelevant.

So, we should add 0.0 price unit to the dictionary d and consequently to the dataframe df.

In [ ]:
for i in range(len(d["Kij"])):
    d["Kij"][i] = np.insert(d["Kij"][i], [0], [0], axis=0)
df = pd.DataFrame(data=d)

##### Diplay the dataframe df

In [ ]:
df.head()

,e,Kij,Hij
0,0,"[0.0, 101.7, 256.8, 283.1, 249.6]","[223.0, 21.6, 748.6, 983.8, 348.0]"
1,1,"[0.0, 103.6, 56.0, 280.4, 86.0]","[181.7, 29.3, 52.5, 628.3, 864.5]"
2,2,"[0.0, 145.0, 68.2, 221.3, 229.2]","[383.9, 476.3, 151.1, 198.2, 167.3]"
3,3,"[0.0, 61.8, 205.6, 54.9, 291.5]","[152.8, 366.5, 133.7, 92.9, 55.6]"
4,4,"[0.0, 200.5, 102.4, 88.0, 184.4]","[189.0, 788.1, 436.4, 608.5, 975.1]"


#### Step 2: towards the fractional knapsack problem

Find the indices in the dataframe of the greedy renovation measures. 

To do so, we calculate $\min_{e}$ for each e building element. We define hereafter some helpers variables and find sort their values.

In [ ]:
indices = []
helper = []
Min = []
for i in df.index:
    product = [km * kg for (km, kg) in zip(df["Kij"][i], df["Hij"][i])]
    indices.append((i, product.index(min([value for value in product if value > 0]))))
    helper.append(i)
for i in df.index:
    k = helper.index(i)
    Min.append((i, np.round(df["Kij"][i][indices[k][1]] * df["Hij"][i][indices[k][1]], decimals=1)))

#### Step 3: sort & reorder

In [ ]:
# sort ascendingly
Min.sort(key=lambda x: x[1])
dt = pd.DataFrame(data=Min)
dt.columns = ["e", "Min_e"]

##### Diplay the dataframe dt

Note that the elements are well ascendingly sorted.

In [ ]:
dt.head()

,e,Min_e
0,0,2196.7
1,1,2940.0
2,3,5100.2
3,2,10305.0
4,6,13368.1


#### Step 4:
Invest Inv starting with the lowest value of $\min_{e}$ with some help-variables

In [ ]:
# Display for the project's inputs:
print("Renovation project's infos:")
print("\tThe number of building elements:\t{}.".format(n))
print("\tThe total renovation investment costs:\t{} price unit.".format(Inv))
# Display for the investement plan:
print("\nInvestement plan with ordered measures:")
count = 0
toInvest = Inv
sum = 0
counter = 0
while toInvest > 0 and count < len(Min):
    flag = helper.index(Min[count][0])
    if toInvest > df["Kij"][Min[count][0]][indices[flag][1]]:
        toInvest -= df["Kij"][Min[count][0]][indices[flag][1]]
        print(
            "\tRenovation measure {}\tKij = {} price unit.\t\tHij = {} price unit.".format(Min[count][0], df["Kij"][Min[count][0]][indices[flag][1]],
                                     df["Hij"][Min[count][0]][indices[flag][1]]))
        sum += df["Kij"][Min[count][0]][indices[flag][1]]
        counter += 1
    else:
        pass
    count += 1
print("\n\tNumber of chosen buiding elements for renovation: {}.".format(counter))
print("\tInvested sum: {} price unit.".format(sum))

Renovation project's infos:
	The number of building elements:	10.
	The total renovation investment costs:	1000 price unit.

Investement plan with ordered measures:
	Renovation measure 0	Kij = 101.7 price unit.		Hij = 21.6 price unit.
	Renovation measure 1	Kij = 56.0 price unit.		Hij = 52.5 price unit.
	Renovation measure 3	Kij = 54.9 price unit.		Hij = 92.9 price unit.
	Renovation measure 2	Kij = 68.2 price unit.		Hij = 151.1 price unit.
	Renovation measure 6	Kij = 96.8 price unit.		Hij = 138.1 price unit.
	Renovation measure 5	Kij = 54.2 price unit.		Hij = 374.7 price unit.
	Renovation measure 9	Kij = 54.4 price unit.		Hij = 391.2 price unit.
	Renovation measure 7	Kij = 182.0 price unit.		Hij = 161.4 price unit.
	Renovation measure 8	Kij = 188.9 price unit.		Hij = 215.7 price unit.
	Renovation measure 4	Kij = 102.4 price unit.		Hij = 436.4 price unit.

	Number of chosen buiding elements for renovation: 10.
	Invested sum: 959.5 price unit.
